In [3]:
%run __init__.ipynb

Successfully connected to MongoDB


In [ ]:
# Variables
DATE_FILTER = yesterday.strftime('%Y%m%d')

In [5]:
# Load Data Source
if use_mongo:
    db_economy = naas_drivers.mongo.get('DB_ALL','yahoo').drop('_id',axis=1)
    db_health = naas_drivers.mongo.get('WORLD_DB_ALL','covid-19').drop('_id',axis=1)

In [8]:
def economy_notif(df):
    # Scope
    cols_to_select = ['DATE_ORDER','TYPE','LABEL','CLOSE','VARV','VARP']
    df = df[cols_to_select]
    df = df[df['DATE_ORDER'] == DATE_FILTER]
    df = df.sort_values(by='DATE_ORDER',ascending=False)
    
    # Get list of top 10 & flop 10 from EQUITIES
    df_equities = df.copy()[(df['TYPE'] == 'EQUITIES')]
    list_mw = df_equities.nlargest(10,'VARP', keep='all')['LABEL'].tolist()
    list_ml = df_equities.nsmallest(10,'VARP', keep='all')['LABEL'].tolist()
    
    # Apply list to main df
    df.loc[:,'TOP'] = np.NaN
    df.loc[df['LABEL'].isin(list_mw),'TOP'] = 'Market winners'
    df.loc[df['LABEL'].isin(list_ml),'TOP'] = 'Market losers'

    return df.reset_index(drop=True)

df_economy = economy_notif(db_economy)
df_economy.head(5)

,DATE_ORDER,TYPE,LABEL,CLOSE,VARV,VARP,TOP
0,20200607,CURRENCIES,USD/IDR,14096.0000,10.599609,0.075252,NaN
1,20200607,COMMODITIES,Silver,17.5700,0.000000,0.000000,NaN
2,20200607,COMMODITIES,Platinum,834.5000,0.000000,0.000000,NaN
3,20200607,COMMODITIES,Natural Gas (HH),1.7800,-0.020000,-1.111111,NaN
4,20200607,COMMODITIES,Copper,2.5505,0.008000,0.314651,NaN


In [4]:
def create_excel_file(file_name,df_eco):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    excel_path = os.path.join(my_repo,excel_name)
    excel_writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
    
    # Save df to excel
    start_row = 4
    df.to_excel(excel_writer,sheet_name =f'ECONOMY', startrow=start_row, index=False)
    add_format_cs(excel_writer,f'{domain_name}',df,start_row)
    
        excel_name = 'WSR_Notifs_' + YESTERDAY.strftime('%Y%m%d')
        excel_path = os.path.join(OUTPUT_FOLDER, excel_name + '.xlsx')
        with pd.ExcelWriter(excel_path) as writer: 
            health.to_excel(writer,sheet_name ="COVID-19")
            economy.to_excel(writer,sheet_name ="ECONOMY")
            
    # Save excel file        
    excel_writer.save()
    
excel_name = f'WSR_Notifs_{DATE_FILTER}'
create_excel_file(excel_name,df_economy)       

In [ ]:
def get_population():
    """
    Get population by countries from GITHUB
    """
    #Boucle pour récupérer la liste des années présentes en colonne dans le fichier
    years = list(map(lambda a : str(a), range(1950, 2020, 1)))
    
    #Colonne à selectionner (*years = selection de toutes les années de la liste years)
    usecols = ["Region, subregion, country or area *", "Country code", "Type", *years]

    # On récupère nos données depuis un fichier Excel
    # usecols nous permet de selectionner uniquement les colonnes dont nous avons besoin
    df = pd.read_excel('https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx',
                        header=16,
                        encoding="utf-8",
                        usecols=usecols)

    # On récupère seulement les valeurs du type "Country/Area"
    df = df[df['Type'] == "Country/Area"]
    
    # récupération des données
    year = str(max(years))
    cols_to_select = ['Region, subregion, country or area *','Country code', year]
    df = df[cols_to_select]
    df = df.rename(index=str,columns = {year:'POPULATION','Region, subregion, country or area *':'COUNTRY'})
    return df.reset_index(drop=True)

df_population = get_population()

In [ ]:
if health_init.empty:
    health_init = pd.read_csv(os.path.join(OUTPUT_FOLDER,'WORLD_DB_ALL.csv'), sep=';',low_memory=False)
    filters = ['WORLDWIDE', 'Europe', 'Asia', 'Africa', 'North America','South America', 'Oceania', 'Other', 'To be affected']
    health_init = health_init[(health_init['DATE_ORDER'] == int(YESTERDAY.strftime('%Y%m%d'))) & (health_init['ENTITY_GROUPS'].isin(filters))]
    health_init = health_init.drop(['Unnamed: 0','VARV','VARP','UNIT_VALUE','UNIT_VAR','UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP','DATE_ORDER','DATE'],axis=1)
    health_init = health_init.reset_index(drop=True)

#Merge REF countries + xls_global
health = health_init.copy()
health = health.merge(REF_COUNTRIES,on='ENTITY',how='left')
health = health.merge(xls_global.drop(['Country code','Region'],axis=1),on='ENTITY_UN',how='left').drop(['ENTITY_UN'],axis=1)

#Population by region
health_region = health.copy()
health_region['ENTITY'] = health_region['ENTITY_GROUPS']
cols_to_keep = ['SCENARIO','ENTITY_GROUPS','ENTITY','KPI']
health_region = health_region.groupby(cols_to_keep).agg({'VALUE':'sum','VALUE_D-1':'sum','POPULATION':'sum'}).reset_index(drop=False)

health = pd.concat([health,health_region],axis=0)
# health = ['SCENARIO','ENTITY_GROUPS','ENTITY','KPI','VALUE','VALUE_D-1','POPULATION']

#Calc 
health['VARV'] = health['VALUE'] - health['VALUE_D-1']
health['VARP'] = health['VARV'] / abs(health['VALUE_D-1']) * 100
health['PVALUE'] = health['VALUE'] / health['POPULATION'] * 100
health['PVALUE_D-1'] = health['VALUE_D-1'] / health['POPULATION'] * 100
health['PVARV'] = health['PVALUE'] - health['PVALUE_D-1']

health.head(5)

In [5]:
# Send email
emails = [
    ('jeremy.ravenel@cashstory.com', 'Jeremy'),
    ('valentin.piquard@cashstory.com', 'Valentin')
]
subject=f"WSR - Daily report {YESTERDAY.strftime('%d/%m/%Y')}"
content=f"<img src=\"{LOGO_CS}\" alt=\"Logo CashStory\" style=\"width: 5%; display: block; margin-right: auto;\" /> \n Hello it's Bob, \n Here is your daily report of your WSR app \n Send from Jupyter with <3"
files=[excel_path]
naas.notifications.send(email_to=emails, subject=subject, html=content, files=files)

Email send
